# Atmospherically Corrected Earth Engine Time Series 

### Overview

This notebook creates an atmospherically corrected Sentinel 2 time series.

### Output

Mean average (cloud-free) pixel values.

#### Initialize

In [ ]:
import ee
ee.Initialize()

In [ ]:
import os
import sys
import numpy as np
from matplotlib import pylab as plt
%matplotlib inline
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'atmcorr'))
import interpolated_lookup_tables as iLUT
from ee_requests import request_meanRadiance
from atmcorr_timeseries import surface_reflectance_timeseries
from plots import plot_timeseries

#### User Input

In [ ]:
# region of interest
geom = ee.Geometry.Point(-122.2586, 37.4967)

# start and end of time series
startDate = ee.Date('1985-01-01')
stopDate  = ee.Date('2020-01-01')

# satellite mission
mission = 'Sentinel2'

#### Interpolated look-up tables

The emulator uses interpolated look-up tables to generate atmospheric correction coeffients.

In [ ]:
iLUTs = iLUT.handler() 
iLUTs.mission = mission
iLUTs.get()

#### Mean Average Radiance

In [ ]:
# earth engine request
request = request_meanRadiance(geom, startDate, stopDate, mission, cloudMask = False)

# get data locally (i.e. client side)
print('Getting data from Earth Engine.. ')
meanRadiance = request.getInfo()
print('Data collection complete')

#### Two or more images?

In [ ]:
num = len(meanRadiance['features'])
print('length of data collection = ',num)
if num < 2:
    print('*'*42+'\nneed at least 2 images, try user inputs?\n'+'*'*42)

#### Atmospheric Correction

In [ ]:
timeSeries = surface_reflectance_timeseries(meanRadiance, iLUTs, mission)

### Plots

#### Visible wavelengths

In [ ]:
print(timeSeries['mission'])

In [ ]:
import plots 
import importlib
importlib.reload(plots)

plot_timeseries(timeSeries, 'visible')

#### Near infrared

In [ ]:
plot_timeseries(timeSeries, 'nir')

#### Short-wave Infrared

In [ ]:
plot_timeseries(timeSeries, 'swir')

#### Atmospheric bands

In [ ]:
plot_timeseries = plots.plot_timeseries
try:
    plot_timeseries(timeSeries, 'atmospheric')
except:
    print('atmospheric bands not available for:',timeSeries['mission'])